In [1]:
import csv
import tensorflow as tf
import numpy as np
import urllib

In [2]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [3]:
class PrintMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        print(f'Epoch {epoch + 1}, Loss: {logs["loss"]}, MAE: {logs["mae"]}')

In [6]:
def sunspot():
    data_url = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/sunspots.csv'
    urllib.request.urlretrieve(data_url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))
            time_step.append(int(row[0]))

    series = np.array(sunspots)


    min_val = np.min(series)
    max_val = np.max(series)
    series -= min_val
    series /= max_val
    time = np.array(time_step)

    split_time = 3000

    time_train = time[:split_time]
    x_train = series[:split_time]
    time_valid = time[split_time:]
    x_valid = series[split_time:]

    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size,
                                 batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10,
                              input_shape=[None, 1],
                              activation="relu"),
        tf.keras.layers.Dense(1)
    ])


    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['mae']
                  )

    model.fit(train_set,
              epochs=50,
              callbacks=[PrintMetricsCallback()])

    return model

In [7]:
if __name__ == '__main__':
    model = sunspot()
    model.save("model.h5")


Epoch 1/50
     90/Unknown - 1s 5ms/step - loss: 0.0109 - mae: 0.0773Epoch 1, Loss: 0.010782803408801556, MAE: 0.07705117762088776
93/93 [==============================] - 1s 5ms/step - loss: 0.0108 - mae: 0.0771
Epoch 2/50
93/93 [==============================] - 1s 5ms/step - loss: 0.0057 - mae: 0.0577
Epoch 3/50
93/93 [==============================] - 1s 4ms/step - loss: 0.0048 - mae: 0.0506
Epoch 4/50
93/93 [==============================] - 1s 5ms/step - loss: 0.0047 - mae: 0.0490
Epoch 5/50
93/93 [==============================] - 1s 5ms/step - loss: 0.0046 - mae: 0.0487
Epoch 6/50
93/93 [==============================] - 1s 5ms/step - loss: 0.0046 - mae: 0.0487
Epoch 7/50
93/93 [==============================] - 1s 4ms/step - loss: 0.0046 - mae: 0.0486
Epoch 8/50
93/93 [==============================] - 1s 5ms/step - loss: 0.0046 - mae: 0.0487
Epoch 9/50
93/93 [==============================] - 1s 5ms/step - loss: 0.0046 - mae: 0.0487
Epoch 10/50
93/93 [========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
